In [ ]:
# Exercise 1 #

import socket

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1) # Allow 1 pending connection

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")

    data = client_socket.recv(1024)
    print(f"Received: {data.decode()}")
    
    # Echo back the data
    client_socket.sendall(b"ACK: " + data)
    client_socket.close()


TCP Server is listening...
Connected to ('127.0.0.1', 64031)
Received: hello


In [ ]:
# Exercise 2 #

import socket

# Create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65432))

print("UDP Server is listening...")

while True:
    data, client_address = server_socket.recvfrom(1024)
    print(f"Received: {data.decode()} from {client_address}")

    # Echo back the data to the client
    server_socket.sendto(b"ACK: " + data, client_address)


UDP Server is listening...
Received: hello from ('127.0.0.1', 57635)


In [ ]:
# Exercise 3 #

import socket

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1) # Allow 1 pending connection

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")

    with open('received_file.txt', 'wb') as f:
        data = client_socket.recv(1024)
        f.write(data)
        print("File received!")
    
    # Echo back the data
    client_socket.sendall(b"ACK: file received")
    client_socket.close()


TCP Server is listening...
Connected to ('127.0.0.1', 50675)
File received!
Connected to ('127.0.0.1', 50679)
File received!
Connected to ('127.0.0.1', 50699)
File received!
Connected to ('127.0.0.1', 50702)
File received!
Connected to ('127.0.0.1', 50703)
File received!
Connected to ('127.0.0.1', 50704)
File received!
Connected to ('127.0.0.1', 50707)
File received!
Connected to ('127.0.0.1', 50708)
File received!
Connected to ('127.0.0.1', 50710)
File received!
Connected to ('127.0.0.1', 50771)
File received!
Connected to ('127.0.0.1', 50773)
File received!
Connected to ('127.0.0.1', 50774)
File received!
Connected to ('127.0.0.1', 50776)
File received!
Connected to ('127.0.0.1', 50777)
File received!
Connected to ('127.0.0.1', 50780)
File received!
Connected to ('127.0.0.1', 50781)
File received!
Connected to ('127.0.0.1', 50783)
File received!
Connected to ('127.0.0.1', 50784)
File received!
Connected to ('127.0.0.1', 50786)
File received!
Connected to ('127.0.0.1', 50787)
File rec

In [ ]:
# Exercise 4 #

import socket

# Create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65432))

print("UDP Server is listening...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    print("File received!")
    print(data)

    with open('received_file.txt', 'wb') as f:
        f.write(data)
        f.close()

    # Echo back the data to the client
    server_socket.sendto(b"ACK: file received", client_address)

UDP Server is listening...
File received!
b"What is Lorem Ipsum?\r\nLorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.\r\n\r\nWhy do we use it?\r\nIt is a long established fact that a reader will be distracted by the readable content of a page when looking at its layout. The point of using Lorem Ipsum is that it has a more-or-less normal distribution of letters, as opposed to using 'Content here, content here', making it look like readable English. Many desktop publishing

#### Problems Encountered:

When making this the first issue I had was that I got an error message. I realised it was because the byte size was too small in 'server_socket.recvfrom(1024)' to receive all the data. So, I increased it to 2048 bytes.

Then the code worked but when I checked the received_file it had no data. I could tell that the 'with open('received_file.txt', 'wb') as f:' line cleared the data already in the received_file but it was unable to successfully write the data to the file. I was sure that the problem was server side as I received a 'File received!' message, which was also echoed to the client. To confirm the problem was server side i printed the data before writing it to the received_file, and the data was printed as expected.

The received_file was definitely being opened because 'with open('received_file.txt', 'wb') as f:' opens the file and clears it. The problem seems to be with the f.write().

After a lot of tinkering, it seems like moving the 'data, client_address = server_socket.recvfrom(2048)' outside the 'with open('received_file.txt', 'wb') as f:' works. One hypothesis is that the having 'data, client_address = server_socket.recvfrom(2048)' in 'with open('received_file.txt', 'wb') as f:' causes 'server_socket.recvfrom(2048)' to be ran repeatedly causing empty data to be fetched from the client which then overwrites the received_file contents.

In [ ]:
# Exercise 5, 6, 7, 8 #

def decrypt(msg, key):
    result= ''

    for char in msg:
      
        if (char.isupper()):
            result += chr((ord(char) - key-65) % 26 + 65)
        elif char.islower():
            result += chr((ord(char) - key - 97) % 26 + 97)
        else:
            result += char
        
    return result

import socket
import threading

def handle_client(client_socket, client_address):
    print(f"Connected to {client_address}")

    while True:
        data = client_socket.recv(1024)  # Receive data from client
        data = data.decode()
        data = decrypt(data, 5)

        if not data:
            print(f"Connection closed by {client_address}")
            break  # If client disconnects or sends nothing, break the loop

        print(f"Received: {data}")

        # Echo back the data
        client_socket.sendall(b"ACK: " + data.encode())

    client_socket.close()
    print(f"Connection with {client_address} closed.")

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)  # Allow up to 5 pending connections

print("TCP Server is listening...")

while True:
    # Accept a new client connection
    client_socket, client_address = server_socket.accept()
    
    # Create a new thread to handle the client
    client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
    client_thread.start()  # Start the thread to handle the client


TCP Server is listening...
Connected to ('127.0.0.1', 49771)
Received: Xpmmzio ozhkzmvopmz vo pid: 9.2°X.


Exception in thread Thread-3 (handle_client):
Traceback (most recent call last):
  File "c:\Users\adidn\miniconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\adidn\miniconda3\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\adidn\miniconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\adidn\AppData\Local\Temp\ipykernel_17548\1304820690.py", line 24, in handle_client
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


Connected to ('127.0.0.1', 49775)
Received: Xpmmzio ozhkzmvopmz vo pid: 9.2°X.
Connection closed by ('127.0.0.1', 49775)
Connection with ('127.0.0.1', 49775) closed.
Connected to ('127.0.0.1', 49784)
Received: Current temperature at uni: 9.2°C.
Connection closed by ('127.0.0.1', 49784)
Connection with ('127.0.0.1', 49784) closed.
